In [19]:
import re
import string
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import nltk
from nltk.corpus import stopwords

import random
from typing import List, Optional

In [20]:
# Zorg dat stopwords beschikbaar zijn
try:
    DUTCH_STOPWORDS = set(stopwords.words("dutch"))
except LookupError:
    nltk.download("stopwords")
    DUTCH_STOPWORDS = set(stopwords.words("dutch"))

# Optioneel: extra domein-specifieke "ruiswoorden"
EXTRA_NOISE = {
    "bij", "voor", "met", "door", "zonder", "over",
    "doelgroep", "werk", "werken", "proces", "praktijk",
    "ontwikkeling", "ontwikkelen", "gaan",
    "leren", "school", "module", "modules", "thema",
    "student", "opleiding", "kun" , "vanuit",
    "eigen", "zelf", "samen", "samenwerken",
    "jaar", "week", "periode", "naasten", "daarnaast",
    "minor", "studenten", "programma", "keuzemodule",
    "casus", "casussen", "cases", "vraagstukken",
    "stage", "stageschool", "kennismakingsstage",
    "kennis", "vaardigheid", "vaardigheden", "ervaring", "ervaringen",
    "lessen", "onderwerpen", "theorie", "praktijk", "praktische", "inhoudelijke",
    "mee", "doe", "vinden", "vind", "kies", "openstaan",
    "belangrijk", "positief", "mogelijkheden","mogelijkheid",
    "impact", "betekenis", "betekent", "betekenen",
    "you", "your", "are", "will", "what", "then", "like", "choose",
    "interested", "experiencing", "hbo", "and", "the", "persoonlijke",
    "learning", "denken", "maken", "business", "verdieping", "emgeving",
    "bouwen", "thinking", "branding", "maken", "urban", "veiligheid",
    "nieuwe", "test", "gebouwde", "concept", "project", "omgeving", "actuele", "acute",
    "yellow", "belt", "serious", "hrm", "mensen", "snel", "binnen", "materialen", 
    "active", "druk", "context", "leven", "complexe" , "brede", "for", "jouw", "manieren"
}


TEXT_STOPWORDS = DUTCH_STOPWORDS | EXTRA_NOISE

PUNCT_TABLE = str.maketrans("", "", string.punctuation + "’‘“”´`")

In [21]:
def clean_text_for_matching(text: str) -> str:
    """
    Maakt tekst klaar voor matching:
    - lowercase
    - verwijder punctuation
    - verwijder cijfers
    - verwijder NL stopwoorden (+ extra noise)
    """
    if not isinstance(text, str):
        return ""
    
    # lowercasing
    text = text.lower()
    
    # punctuation verwijderen
    text = text.translate(PUNCT_TABLE)
    
    # cijfers eruit
    text = re.sub(r"\d+", " ", text)
    
    # meerdere spaties
    text = re.sub(r"\s+", " ", text).strip()
    
    # stopwoorden filteren
    tokens = [
        tok for tok in text.split()
        if tok not in TEXT_STOPWORDS and len(tok) > 2
    ]
    
    return " ".join(tokens)


In [39]:
# Voorbeeld: pas dit pad aan naar jouw echte bron
df = pd.read_csv("Uitgebreide_VKM_dataset_cleaned2.csv")  # of je eigen load-functie

# Verwachte kolommen (minimaal):
# id, name, shortdescription, location, studycredit, level, module_tags, ...
# We maken/overschrijven een 'clean_text'-kolom op basis van relevante velden.

TEXT_COLUMNS = ["name", "shortdescription"]  # breid uit als je meer wilt meenemen

def build_raw_text(row: pd.Series) -> str:
    parts = []
    for col in TEXT_COLUMNS:
        if col in row and isinstance(row[col], str):
            parts.append(row[col])
    return " ".join(parts)

df["raw_text"] = df.apply(build_raw_text, axis=1)
df["clean_text"] = df["raw_text"].apply(clean_text_for_matching)



In [ ]:
# Vectorizer eenmalig fitten op de complete dataset
vectorizer = TfidfVectorizer(
        ngram_range=(1, 2),
        max_df=0.8,      # woorden die in >80% van de modules zitten eruit
        min_df=2,        # woorden die maar 1x voorkomen eruit
)
X = vectorizer.fit_transform(df["clean_text"])

FEATURE_NAMES = vectorizer.get_feature_names_out()
X.shape, len(FEATURE_NAMES)


((207, 1336), 1336)

In [24]:
def _format_term_list(terms: List[str]) -> str:
    """
    Maak een nette NL opsomming:
    - 'A'
    - 'A' en 'B'
    - 'A', 'B' en 'C'
    """
    if not terms:
        return ""
    if len(terms) == 1:
        return f"'{terms[0]}'"
    if len(terms) == 2:
        return f"'{terms[0]}' en '{terms[1]}'"
    # 3 of meer
    quoted = [f"'{t}'" for t in terms]
    hoofd = ", ".join(quoted[:-1])
    laatste = quoted[-1]
    return f"{hoofd} en {laatste}"


In [25]:
def build_reason(match_terms: List[str], module_name: Optional[str] = None, score: Optional[float] = None) -> str:
    """
    Genereer een korte NL tekst waarom de module past.
    - match_terms: termen waarop je matcht (interesses / keywords)
    - module_name: optioneel, naam van de module voor iets persoonlijkere tekst
    - score: optioneel, genormaliseerde similarity-score (0–1)
    """
    terms_str = _format_term_list(match_terms)

    # score -> indicatie hoe sterk de match is
    if score is None:
        kwalificatie = "goed"
    elif score >= 0.8:
        kwalificatie = "erg goed"
    elif score >= 0.6:
        kwalificatie = "goed"
    else:
        kwalificatie = "redelijk"

    # Geen specifieke termen: algemene uitleg
    if not match_terms:
        templates = [
            "Deze module sluit {kwalificatie} aan bij je interesses op basis van tekstuele overeenkomsten.",
            "Op basis van de overeenkomst tussen jouw profiel en de modulebeschrijving lijkt deze module {kwalificatie} bij je te passen.",
            "Deze module lijkt inhoudelijk {kwalificatie} aan te sluiten bij wat je interessant vindt."
        ]
    else:
        # Met match_terms
        if module_name:
            templates = [
                "Je interesse in {terms} komt duidelijk terug in '{module}', waardoor deze module {kwalificatie} bij je aansluit.",
                "Omdat {terms} centraal staan in '{module}', past deze module {kwalificatie} bij jouw interesses.",
                "In '{module}' komen {terms} aan bod, wat goed aansluit bij jouw interesses."
            ]
        else:
            templates = [
                "Deze module sluit {kwalificatie} aan bij je interesses in {terms}.",
                "Omdat {terms} in deze module aan bod komen, lijkt deze {kwalificatie} bij je te passen.",
                "Je interesse in {terms} komt terug in de inhoud van deze module, waardoor deze goed bij je past."
            ]

    template = random.choice(templates)
    return template.format(
        kwalificatie=kwalificatie,
        terms=terms_str,
        module=module_name if module_name else ""
    )


In [26]:
def extract_match_terms(student_vec, module_vec, feature_names, max_terms: int = 8) -> List[str]:
    """
    Bepaal welke termen (woorden) zowel in de studentvector
    als in de modulevector voorkomen.
    """
    # indices waar de vector niet 0 is
    student_idx = set(student_vec.nonzero()[1])
    module_idx = set(module_vec.nonzero()[1])

    shared_idx = sorted(student_idx & module_idx)
    terms = [feature_names[i] for i in shared_idx]

    return terms[:max_terms]


In [27]:
def recommend_modules(student_profile: str, top_n: int = 5, studycredit: Optional[int] = None, level: Optional[str] = None, location: Optional[str] = None) -> pd.DataFrame:
    """
    Geeft top N modules voor een studentprofiel,
    inclusief:
    - similarity (genormaliseerd 0–1 binnen de selectie)
    - similarity_raw (werkelijke cosine score)
    - match_terms
    - reason (NL-uitleg)
    """

    # Start met volledige df
    filtered_df = df.copy()

    # Filter op studiepunten, niveau, locatie
    if studycredit is not None:
        filtered_df = filtered_df[filtered_df["studycredit"] == studycredit]

    if level is not None:
        filtered_df = filtered_df[filtered_df["level"].isin([level])]

    if location is not None:
        filtered_df = filtered_df[
            filtered_df["location"].str.contains(location, case=False, na=False)
        ]

    # Niks over? Return lege tabel met juiste kolommen
    if filtered_df.empty:
        return pd.DataFrame(
            columns=[
                "id", "name", "shortdescription", "similarity",
                "similarity_raw", "location", "studycredit",
                "level", "module_tags", "match_terms", "reason"
            ]
        )

    # Studentprofiel vectoriseren
    clean_profile = clean_text_for_matching(student_profile)
    student_vec = vectorizer.transform([clean_profile])

    # Bijbehorende rijen uit X pakken
    row_indices = filtered_df.index.to_numpy()
    X_filtered = X[row_indices]

    # Cosine similarity
    sims = cosine_similarity(student_vec, X_filtered).flatten()

    # In kopie wegschrijven
    filtered_df = filtered_df.copy()
    filtered_df["similarity_raw"] = sims

    # Normaliseren naar 0–1 binnen deze selectie
    max_sim = sims.max()
    if max_sim > 0:
        norm_sims = sims / max_sim
    else:
        norm_sims = sims  # alles 0

    filtered_df["similarity"] = norm_sims

    # Sorteren en top N pakken
    top = filtered_df.sort_values("similarity", ascending=False).head(top_n)

    match_terms_list: List[List[str]] = []
    reasons: List[str] = []

    for idx in top.index:
        # positie van deze rij binnen X_filtered
        pos = np.where(row_indices == idx)[0][0]
        module_vec = X_filtered[pos]

        terms = extract_match_terms(student_vec, module_vec, FEATURE_NAMES)
        match_terms_list.append(terms)

        reasons.append(
            build_reason(
                terms,
                module_name=top.at[idx, "name"],
                score=top.at[idx, "similarity"]  # genormaliseerde score
            )
        )

    # Kolommen selecteren en verrijken
    top = top[["id", "name", "shortdescription", "similarity", "similarity_raw", "location", "studycredit", "level", "module_tags"]].copy()
    top["match_terms"] = match_terms_list
    top["reason"] = reasons

    return top.reset_index(drop=True)


In [32]:
sam_profiel = """
Ik heb een achtergrond in de zorg en ik ben geïnteresseerd in hoe technologie kan helpen om zorg en welzijn slimmer te organiseren. 
Dingen als smart health, e-health, zorg op afstand en innovatieve oplossingen voor ouderen spreken me erg aan. 
Ik werk graag met praktische toepassingen van technologie in de zorgpraktijk en vind het leuk om in teams aan echte cases te werken.
"""
pd.set_option("display.max_colwidth", None)

result = recommend_modules(
    student_profile=sam_profiel,
    top_n=5,
    # studycredit=15,      # of None
    # level="NLQF5",       # of None
    # location="Breda"     # of None
)

result


,id,name,shortdescription,similarity,similarity_raw,location,studycredit,level,module_tags,match_terms,reason
0,171,Technologie in zorg en welzijn,"Technologie, zorg, welzijn",1.000000,0.719090,Den Bosch,30,NLQF6,"['technologie', 'zorg', 'welzijn']","[technologie, welzijn, zorg]","Je interesse in 'technologie', 'welzijn' en 'zorg' komt duidelijk terug in 'Technologie in zorg en welzijn', waardoor deze module erg goed bij je aansluit."
1,176,Technologie die ècht werkt: innovatie in zorg en welzijn,"Technologie, smart health, robotica, zorg, welzijn",0.889463,0.639604,Breda,15,NLQF5,"['technologie', 'smart', 'health', 'robotica', 'zorg', 'welzijn']","[health, smart, technologie, welzijn, zorg]","In 'Technologie die ècht werkt: innovatie in zorg en welzijn' komen 'health', 'smart', 'technologie', 'welzijn' en 'zorg' aan bod, wat goed aansluit bij jouw interesses."
2,172,Zorg dichtbij,"Zorg op afstand, technologie, implementeren, zorg, veranderkunde",0.836704,0.601665,Breda,15,NLQF5,"['zorg', 'afstand', 'technologie', 'implementeren', 'veranderkunde']","[afstand, technologie, zorg]","Je interesse in 'afstand', 'technologie' en 'zorg' komt duidelijk terug in 'Zorg dichtbij', waardoor deze module erg goed bij je aansluit."
3,199,Acuut Complexe Zorg,"Zorg, welzijn, acute zorg",0.700425,0.503669,Breda,30,NLQF6,"['zorg', 'welzijn']","[welzijn, zorg]","Je interesse in 'welzijn' en 'zorg' komt duidelijk terug in 'Acuut Complexe Zorg', waardoor deze module goed bij je aansluit."
4,320,Smart Health,Smart Health,0.462715,0.332733,Breda,15,NLQF5,"['smart', 'health']","[health, smart]","Je interesse in 'health' en 'smart' komt duidelijk terug in 'Smart Health', waardoor deze module redelijk bij je aansluit."


In [34]:
gerda_profiel = """
Ik ben vooral geïnteresseerd in data, kunstmatige intelligentie en slimme systemen. 
Ik vind onderwerpen zoals machine learning, big data, Internet of Things, smart industry en automatisering supertof. 
Ik werk graag analytisch, met datasets en tools, en ik vind het leuk om oplossingen te bouwen die processen slimmer maken.
"""
pd.set_option("display.max_colwidth", None)

result = recommend_modules(
    student_profile=gerda_profiel,
    top_n=5,
    # studycredit=15,      # of None
    # level="NLQF5",       # of None
    # location="Breda"     # of None
)

result

,id,name,shortdescription,similarity,similarity_raw,location,studycredit,level,module_tags,match_terms,reason
0,333,Smart Industry & Internet of Things,"big data, smart robotics, Artificial Intelligence, machine learning, IoT, smart industry, Industry 4.0",1.000000,0.497496,Breda,30,NLQF6,"['big', 'data', 'smart', 'robotics', 'artificial', 'intelligence', 'machine', 'iot', 'industry', '40']","[big, data, industry, internet, machine, smart, things]","Omdat 'big', 'data', 'industry', 'internet', 'machine', 'smart' en 'things' centraal staan in 'Smart Industry & Internet of Things', past deze module erg goed bij jouw interesses."
1,255,Module 2.3 Data gestuurd Ontwerp (Data Driven Design),"Data driven design, big data, BIM/GIS, parametrisch ontwerpen, virtual reality, storytelling",0.496040,0.246778,Den Bosch en Tilburg,15,NLQF5,"['data', 'driven', 'design', 'big', 'bimgis', 'parametrisch', 'ontwerpen', 'virtual', 'reality', 'storytelling']","[big, data]","Omdat 'big' en 'data' centraal staan in 'Module 2.3 Data gestuurd Ontwerp (Data Driven Design)', past deze module redelijk bij jouw interesses."
2,319,Green Belt,Smart Industry / Smart Health,0.412465,0.205200,Tilburg,15,NLQF5,"['smart', 'industry', 'health']","[industry, smart]","Je interesse in 'industry' en 'smart' komt duidelijk terug in 'Green Belt', waardoor deze module redelijk bij je aansluit."
3,324,Engineering for non-engineers,Smart Industry / Smart Health / Smart Energy,0.388844,0.193448,Breda,15,NLQF5,"['smart', 'industry', 'health', 'energy']","[industry, smart]","Omdat 'industry' en 'smart' centraal staan in 'Engineering for non-engineers', past deze module redelijk bij jouw interesses."
4,353,Data/Visualisatie/Automatisering (Tech & Data) (CT),"Datavaardigheid, automatisering, programeren, sensoriek",0.384479,0.191277,Breda en Den Bosch,15,NLQF5,"['datavaardigheid', 'automatisering', 'programeren', 'sensoriek']","[automatisering, data]","Omdat 'automatisering' en 'data' centraal staan in 'Data/Visualisatie/Automatisering (Tech & Data) (CT)', past deze module redelijk bij jouw interesses."


In [36]:
klaas_profiel = """
Ik vind innovatie en vernieuwing binnen organisaties heel interessant. 
Ik werk graag aan nieuwe ideeën, producten en diensten en denk na over hoe je die in de praktijk kunt implementeren. 
Thema’s zoals innovatiemanagement, verandermanagement, business modellen en het verbeteren van processen spreken me erg aan. 
Ik ben minder bezig met diep technische details, maar meer met strategie en organisatie.

"""
pd.set_option("display.max_colwidth", None)

result = recommend_modules(
    student_profile=klaas_profiel,
    top_n=5,
    # studycredit=15,      # of None
    # level="NLQF5",       # of None
    # location="Breda"     # of None
)

result

,id,name,shortdescription,similarity,similarity_raw,location,studycredit,level,module_tags,match_terms,reason
0,331,Innovatiemanagement,"Het centrale thema van deze minor heeft betrekking op technologie en innovatie van nieuwe producten/diensten en processen binnen bestaande en nieuwe organisatie en de mogelijkheden van innovatie om verbeteringen in de bedrijfsvoering van organisaties door te voeren. Innovatie of vernieuwing is het invoeren van nieuwe ideeën, goederen, diensten en processen. Innovatie kan plaatsvinden binnen organisaties maar ook binnen bredere verbanden. Het proces van innovatie draait om dingen op een nieuwe (en zo mogelijk ook betere) manier aan te pakken. Het centrale thema van deze minor heeft betrekking op technologie en innovatie van nieuwe producten/diensten en processen binnen bestaande en nieuwe organisatie en de mogelijkheden van innovatie om verbeteringen in de bedrijfsvoering van organisaties door te voeren. Innovatie of vernieuwing is het invoeren van nieuwe ideeën, goederen, diensten en processen. Innovatie kan plaatsvinden binnen organisaties maar ook binnen bredere verbanden. Het proces van innovatie draait om dingen op een nieuwe (en zo mogelijk ook betere) manier aan te pakken.",1.000000,0.431581,Tilburg,30,NLQF6,"['innovatie draait', 'innovatie plaatsvinden', 'innovatie verbeteringen', 'innovatie', 'processen innovatie', 'innovatie vernieuwing', 'voeren innovatie', 'vernieuwing invoeren']","[diensten, ideeën, innovatie, innovatiemanagement, organisatie, organisaties, processen, vernieuwing]","In 'Innovatiemanagement' komen 'diensten', 'ideeën', 'innovatie', 'innovatiemanagement', 'organisatie', 'organisaties', 'processen' en 'vernieuwing' aan bod, wat goed aansluit bij jouw interesses."
1,272,Duurzame Business Modellen,"Duurzaamheid, business modellen, advisering",0.472193,0.203790,Den Bosch,15,NLQF5,"['duurzaamheid', 'modellen', 'advisering']",[modellen],"In 'Duurzame Business Modellen' komen 'modellen' aan bod, wat goed aansluit bij jouw interesses."
2,327,Analyseren van software en infrastructuur (1.3 BIM),"In deze module leer je de software en infrastructuur inrichting van een organisatie kennen adviseer je een organisatie bij de aanschaf van een nieuw (software) pakket alsmede de impact die dit heeft op de bestaande organisatie (processen, infrastructuur en andere software). In deze module leer je de software en infrastructuur inrichting van een organisatie kennen adviseer je een organisatie bij de aanschaf van een nieuw (software) pakket alsmede de impact die dit heeft op de bestaande organisatie (processen, infrastructuur en andere software).",0.372361,0.160704,Breda,15,NLQF5,"['software infrastructuur', 'infrastructuur software', 'software', 'leer software', 'software pakket', 'adviseer organisatie', 'processen infrastructuur', 'nieuw software', 'inrichting organisatie']","[organisatie, processen]","In 'Analyseren van software en infrastructuur (1.3 BIM)' komen 'organisatie' en 'processen' aan bod, wat goed aansluit bij jouw interesses."
3,235,BI Innovatie,"innovatie, innovatie methoden, casework, business models, responsible",0.289160,0.124796,Den Bosch,15,NLQF5,"['innovatie', 'methoden', 'casework', 'models', 'responsible']",[innovatie],"In 'BI Innovatie' komen 'innovatie' aan bod, wat goed aansluit bij jouw interesses."
4,181,Management in de zorg,"Bij de keuzemodule Management in de Zorg leer je over organisatie-, personeels- en kwaliteitsmanagement in de zorgsector. Ook strategisch management, kwaliteitsmanagement en ondernemen in de zorg komen aan bod. Praktische vaardigheden zoals communiceren en presenteren worden ook behandeld, evenals het bevorderen van samenwerking tussen betrokkenen, afdelingen en organisaties. Management in de zorg",0.262773,0.113408,Breda,15,NLQF5,"['management kwaliteitsmanagement', 'personeels kwaliteitsmanagement', 'kwaliteitsmanagement zorgsector', 'kwaliteitsmanagement ondernemen', 'management', 'kwaliteitsmanagement', 'management zorg', 'organisa

In [37]:
mathilda_profiel = """
Ik vind het leuk om creatieve oplossingen te ontwerpen met behulp van technologie. 
Onderwerpen zoals data driven design, UX/UI, virtual reality, storytelling en het visueel maken van informatie spreken me erg aan. 
Ik wil graag leren hoe je technieken zoals data-analyse en visualisatie inzet om betere ontwerpen en gebruikservaringen te maken.
"""
pd.set_option("display.max_colwidth", None)

result = recommend_modules(
    student_profile=mathilda_profiel,
    top_n=5,
    # studycredit=15,      # of None
    # level="NLQF5",       # of None
    # location="Breda"     # of None
)

result

,id,name,shortdescription,similarity,similarity_raw,location,studycredit,level,module_tags,match_terms,reason
0,255,Module 2.3 Data gestuurd Ontwerp (Data Driven Design),"Data driven design, big data, BIM/GIS, parametrisch ontwerpen, virtual reality, storytelling",1.000000,0.438477,Den Bosch en Tilburg,15,NLQF5,"['data', 'driven', 'design', 'big', 'bimgis', 'parametrisch', 'ontwerpen', 'virtual', 'reality', 'storytelling']","[data, design, driven, ontwerpen, reality, storytelling, virtual]","Omdat 'data', 'design', 'driven', 'ontwerpen', 'reality', 'storytelling' en 'virtual' centraal staan in 'Module 2.3 Data gestuurd Ontwerp (Data Driven Design)', past deze module erg goed bij jouw interesses."
1,321,Virtual en Augmented Reality,"Virtual Reality, Augmented Reality, Digital Twin, Serious Game, User Experience",0.535943,0.234999,Breda,15,NLQF5,"['virtual', 'reality', 'augmented', 'digital', 'twin', 'game', 'user', 'experience']","[reality, virtual]","Omdat 'reality' en 'virtual' centraal staan in 'Virtual en Augmented Reality', past deze module redelijk bij jouw interesses."
2,174,Mensgericht ontwerpen,"Design thinking, service design, technolgie, UX-design, doelgroep",0.427740,0.187554,Breda,15,NLQF5,"['design', 'service', 'technolgie', 'uxdesign']","[design, ontwerpen]","Je interesse in 'design' en 'ontwerpen' komt duidelijk terug in 'Mensgericht ontwerpen', waardoor deze module redelijk bij je aansluit."
3,254,Module 2.1/2.3 Toekomstgericht Ontwerpen,"Natuurinclusief Ontwerpen, post humaan denken, transitiedenken, paradigma shift",0.380953,0.167039,Den Bosch en Tilburg,15,NLQF5,"['natuurinclusief', 'ontwerpen', 'post', 'humaan', 'transitiedenken', 'paradigma', 'shift']",[ontwerpen],"In 'Module 2.1/2.3 Toekomstgericht Ontwerpen' komen 'ontwerpen' aan bod, wat goed aansluit bij jouw interesses."
4,171,Technologie in zorg en welzijn,"Technologie, zorg, welzijn",0.343151,0.150464,Den Bosch,30,NLQF6,"['technologie', 'zorg', 'welzijn']",[technologie],"Omdat 'technologie' centraal staan in 'Technologie in zorg en welzijn', past deze module redelijk bij jouw interesses."


In [42]:
from typing import List, Optional
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def recommend_modules_with(
    df: pd.DataFrame,
    vectorizer,
    X,
    feature_names,
    student_profile: str,
    top_n: int = 5,
    studycredit: Optional[int] = None,
    level: Optional[str] = None,
    location: Optional[str] = None
) -> pd.DataFrame:
    """
    Zelfde idee als recommend_modules, maar gebruikt de meegegeven
    vectorizer, X en feature_names. Handig voor experimenten met TF-IDF.
    """
    filtered_df = df.copy()

    # Filter op studiepunten, niveau, locatie
    if studycredit is not None:
        filtered_df = filtered_df[filtered_df["studycredit"] == studycredit]

    if level is not None:
        filtered_df = filtered_df[filtered_df["level"].isin([level])]

    if location is not None:
        filtered_df = filtered_df[
            filtered_df["location"].str.contains(location, case=False, na=False)
        ]

    # Niks over? -> lege tabel terug
    if filtered_df.empty:
        return pd.DataFrame(
            columns=[
                "id", "name", "shortdescription", "similarity",
                "similarity_raw", "location", "studycredit",
                "level", "module_tags", "match_terms", "reason"
            ]
        )

    # Studentprofiel vectoriseren
    clean_profile = clean_text_for_matching(student_profile)
    student_vec = vectorizer.transform([clean_profile])

    # Bijbehorende rijen uit X pakken
    row_indices = filtered_df.index.to_numpy()
    X_filtered = X[row_indices]

    # Cosine similarity
    sims = cosine_similarity(student_vec, X_filtered).flatten()

    filtered_df = filtered_df.copy()
    filtered_df["similarity_raw"] = sims

    # Normaliseren naar 0–1 binnen deze selectie
    max_sim = sims.max()
    if max_sim > 0:
        norm_sims = sims / max_sim
    else:
        norm_sims = sims

    filtered_df["similarity"] = norm_sims

    # Sorteren en top N pakken
    top = filtered_df.sort_values("similarity", ascending=False).head(top_n)

    match_terms_list: List[List[str]] = []
    reasons: List[str] = []

    for idx in top.index:
        # positie van deze rij binnen X_filtered
        pos = np.where(row_indices == idx)[0][0]
        module_vec = X_filtered[pos]

        terms = extract_match_terms(student_vec, module_vec, feature_names)
        match_terms_list.append(terms)

        reasons.append(
            build_reason(
                match_terms=terms,
                module_name=top.at[idx, "name"],
                score=top.at[idx, "similarity"]
            )
        )

    top = top[[
        "id",
        "name",
        "shortdescription",
        "similarity",
        "similarity_raw",
        "location",
        "studycredit",
        "level",
        "module_tags",
    ]].copy()

    top["match_terms"] = match_terms_list
    top["reason"] = reasons

    return top.reset_index(drop=True)

In [41]:
tfidf_configs = {
    "baseline": TfidfVectorizer(),
    "ngrams_1_2": TfidfVectorizer(ngram_range=(1, 2)),
    "pruned_vocab": TfidfVectorizer(
        ngram_range=(1, 2),
        max_df=0.8,      # woorden die in >80% van de modules zitten eruit
        min_df=2,        # woorden die maar 1x voorkomen eruit
    ),
    "sublinear": TfidfVectorizer(
        ngram_range=(1, 2),
        sublinear_tf=True
    ),
}

In [43]:
student_profile = """
Ik heb een achtergrond in de zorg en ik ben geïnteresseerd in hoe technologie kan helpen om zorg en welzijn slimmer te organiseren. 
Dingen als smart health, e-health, zorg op afstand en innovatieve oplossingen voor ouderen spreken me erg aan. 
Ik werk graag met praktische toepassingen van technologie in de zorgpraktijk en vind het leuk om in teams aan echte cases te werken.
"""

for name, vec in tfidf_configs.items():
    print("=== Config:", name, "===\n")

    # Vectorizer fitten op de clean_text van alle modules
    X_exp = vec.fit_transform(df["clean_text"])
    feature_names = vec.get_feature_names_out()

    # Aanbevelingen genereren met deze vectorizer
    result = recommend_modules_with(
        df=df,
        vectorizer=vec,
        X=X_exp,
        feature_names=feature_names,
        student_profile=student_profile,
        top_n=5,
        # studycredit=15,       # pas aan of zet op None
        # level="NLQF5",        # pas aan of zet op None
        # location="Breda"      # pas aan of zet op None
    )

    display(result[["name", "similarity", "similarity_raw", "match_terms", "reason"]])
    print("\n\n")

=== Config: baseline ===



,name,similarity,similarity_raw,match_terms,reason
0,Technologie in zorg en welzijn,1.000000,0.719090,"[technologie, welzijn, zorg]","In 'Technologie in zorg en welzijn' komen 'technologie', 'welzijn' en 'zorg' aan bod, wat goed aansluit bij jouw interesses."
1,Technologie die ècht werkt: innovatie in zorg en welzijn,0.889463,0.639604,"[health, smart, technologie, welzijn, zorg]","Je interesse in 'health', 'smart', 'technologie', 'welzijn' en 'zorg' komt duidelijk terug in 'Technologie die ècht werkt: innovatie in zorg en welzijn', waardoor deze module erg goed bij je aansluit."
2,Zorg dichtbij,0.836704,0.601665,"[afstand, technologie, zorg]","Je interesse in 'afstand', 'technologie' en 'zorg' komt duidelijk terug in 'Zorg dichtbij', waardoor deze module erg goed bij je aansluit."
3,Acuut Complexe Zorg,0.700425,0.503669,"[welzijn, zorg]","Je interesse in 'welzijn' en 'zorg' komt duidelijk terug in 'Acuut Complexe Zorg', waardoor deze module goed bij je aansluit."
4,Smart Health,0.462715,0.332733,"[health, smart]","Omdat 'health' en 'smart' centraal staan in 'Smart Health', past deze module redelijk bij jouw interesses."





=== Config: ngrams_1_2 ===



,name,similarity,similarity_raw,match_terms,reason
0,Technologie in zorg en welzijn,1.000000,0.513428,"[technologie, welzijn, zorg, zorg welzijn]","Omdat 'technologie', 'welzijn', 'zorg' en 'zorg welzijn' centraal staan in 'Technologie in zorg en welzijn', past deze module erg goed bij jouw interesses."
1,Technologie die ècht werkt: innovatie in zorg en welzijn,0.990917,0.508765,"[health, smart, smart health, technologie, welzijn, zorg, zorg welzijn]","Omdat 'health', 'smart', 'smart health', 'technologie', 'welzijn', 'zorg' en 'zorg welzijn' centraal staan in 'Technologie die ècht werkt: innovatie in zorg en welzijn', past deze module erg goed bij jouw interesses."
2,Zorg dichtbij,0.907509,0.465941,"[afstand, technologie, zorg, zorg afstand]","Omdat 'afstand', 'technologie', 'zorg' en 'zorg afstand' centraal staan in 'Zorg dichtbij', past deze module erg goed bij jouw interesses."
3,Acuut Complexe Zorg,0.773606,0.397191,"[welzijn, zorg, zorg welzijn]","Je interesse in 'welzijn', 'zorg' en 'zorg welzijn' komt duidelijk terug in 'Acuut Complexe Zorg', waardoor deze module goed bij je aansluit."
4,Smart Health,0.699065,0.358920,"[health, smart, smart health]","Je interesse in 'health', 'smart' en 'smart health' komt duidelijk terug in 'Smart Health', waardoor deze module goed bij je aansluit."





=== Config: pruned_vocab ===



,name,similarity,similarity_raw,match_terms,reason
0,Technologie die ècht werkt: innovatie in zorg en welzijn,1.000000,0.873147,"[health, smart, smart health, technologie, welzijn, zorg, zorg welzijn]","Omdat 'health', 'smart', 'smart health', 'technologie', 'welzijn', 'zorg' en 'zorg welzijn' centraal staan in 'Technologie die ècht werkt: innovatie in zorg en welzijn', past deze module erg goed bij jouw interesses."
1,Technologie in zorg en welzijn,0.877949,0.766579,"[technologie, welzijn, zorg, zorg welzijn]","Je interesse in 'technologie', 'welzijn', 'zorg' en 'zorg welzijn' komt duidelijk terug in 'Technologie in zorg en welzijn', waardoor deze module erg goed bij je aansluit."
2,Zorg dichtbij,0.799766,0.698313,"[technologie, zorg]","Je interesse in 'technologie' en 'zorg' komt duidelijk terug in 'Zorg dichtbij', waardoor deze module goed bij je aansluit."
3,Langer thuis in de wijk,0.672773,0.587430,"[welzijn, zorg, zorg welzijn]","Je interesse in 'welzijn', 'zorg' en 'zorg welzijn' komt duidelijk terug in 'Langer thuis in de wijk', waardoor deze module goed bij je aansluit."
4,Acuut Complexe Zorg,0.600137,0.524008,"[welzijn, zorg, zorg welzijn]","Je interesse in 'welzijn', 'zorg' en 'zorg welzijn' komt duidelijk terug in 'Acuut Complexe Zorg', waardoor deze module goed bij je aansluit."





=== Config: sublinear ===



,name,similarity,similarity_raw,match_terms,reason
0,Technologie in zorg en welzijn,1.000000,0.472889,"[technologie, welzijn, zorg, zorg welzijn]","Omdat 'technologie', 'welzijn', 'zorg' en 'zorg welzijn' centraal staan in 'Technologie in zorg en welzijn', past deze module erg goed bij jouw interesses."
1,Technologie die ècht werkt: innovatie in zorg en welzijn,0.975648,0.461373,"[health, smart, smart health, technologie, welzijn, zorg, zorg welzijn]","Je interesse in 'health', 'smart', 'smart health', 'technologie', 'welzijn', 'zorg' en 'zorg welzijn' komt duidelijk terug in 'Technologie die ècht werkt: innovatie in zorg en welzijn', waardoor deze module erg goed bij je aansluit."
2,Zorg dichtbij,0.840290,0.397364,"[afstand, technologie, zorg, zorg afstand]","Omdat 'afstand', 'technologie', 'zorg' en 'zorg afstand' centraal staan in 'Zorg dichtbij', past deze module erg goed bij jouw interesses."
3,Smart Health,0.820923,0.388205,"[health, smart, smart health]","Je interesse in 'health', 'smart' en 'smart health' komt duidelijk terug in 'Smart Health', waardoor deze module erg goed bij je aansluit."
4,Acuut Complexe Zorg,0.633078,0.299376,"[welzijn, zorg, zorg welzijn]","Je interesse in 'welzijn', 'zorg' en 'zorg welzijn' komt duidelijk terug in 'Acuut Complexe Zorg', waardoor deze module goed bij je aansluit."
